In [6]:
import pandas as pd
import sqlite3

#read the CSV
df = pd.read_csv('compiled_data.csv')
#connect to a database
conn = sqlite3.connect("policeData.db") #if the db does not exist, this creates a policeData.db file in the current directory
#store your table in the database:
df.to_sql('data', conn)

In [10]:
df

,opened,case,county,agency,uniformed-plainClothes,officersInjured,officersKilled,suspectsInjured,suspectsKilled,offense,officer-suspectEthnicity,time,SLEDInvestigation,currentStatus
0,1/3/18,31-18-0001,DORCHESTER,NORTH CHARLESTON PD,UNIFORMED,NaN,NaN,1,NaN,ARMED ROBBERY,NaN,NaN,NaN,NaN
1,1/11/18,32-18-0005,LEXINGTON,LEXINGTON COUNTY SO,UNIFORMED,NaN,NaN,NaN,1,TRAFFIC STOP,NaN,NaN,NaN,NaN
2,1/16/18,34-18-0011,YORK,YORK COUNTY SO / YORK PD,UNIFORMED,3,1,1,NaN,DOMESTIC,NaN,NaN,NaN,NaN
3,1/21/18,34-18-0013,SPARTANBURG,LAURENS COUNTY SO,UNIFORMED,NaN,NaN,1,NaN,STOLEN VEHICLE,NaN,NaN,NaN,NaN
4,1/24/18,34-18-0016,SPARTANBURG,GREER PD,UNIFORMED,1,NaN,1,NaN,SUSPICIOUS VEHICLE,NaN,NaN,NaN,NaN
5,2/15/18,34-18-0029,ANDERSON,ANDERSON COUNTY SO,UNIFORMED,NaN,NaN,1,NaN,TRAFFIC STOP,NaN,NaN,NaN,NaN
6,2/27/18,32-18-0022,CHESTER,CHESTER COUNTY SO / CHESTER PD,UNIFORMED,NaN,NaN,NaN,1,STOLEN VEHICLE,NaN,NaN,NaN,NaN
7,3/2/18,31-18-0015,BERKELEY,BERKELEY COUNTY SO,UNIFORMED,1,NaN,1,NaN,DOMESTIC,NaN,NaN,NaN,NaN
8,3/15/18,32-18-0032,AIKEN,AIKEN COUNTY SO,UNIFORMED,NaN,NaN,1,NaN,MENTAL SUBJECT,NaN,NaN,NaN,NaN
9,3/19/18,34-18-0051,GREENVILLE,GREENVILLE COUNTY SO,UNIFORMED,NaN,NaN,NaN,1,SUICIDAL SUBJECT,NaN,NaN,NaN,NaN


In [12]:
pd.read_sql_query("select agency, count(agency) from data group by agency", conn)

,agency,count(agency)
0,None,0
1,(DEA *****),1
2,AIKEN CO SO,3
3,AIKEN CO SO & ATF,1
4,AIKEN COUNTY SO,4
5,AIKEN DEPT. PUB. SAFETY,4
6,AIKEN DPS,1
7,AIKEN DPS / Aiken County SO / SLED,1
8,AIKEN SO,1
9,ALLENDALE PD,2


In [15]:
pd.read_sql_query("select county, count(suspectsKilled) from data group by county", conn)

,county,count(suspectsKilled)
0,(ORANGEBURG),1
1,AIKEN,7
2,ALLENDALE,0
3,ANDERSON,13
4,BARNWELL,1
5,BEAUFORT,1
6,BERKELEY,5
7,BERKELY,1
8,CALHOUN,0
9,CHARLESTON,16
